# Deep Learning in Medicine
### BMSC-GA 4493, BMIN-GA 3007 
### Lab 2: PyTorch Tutorial and Loss Functions


### Goal of this lab: 
    - Understand Pytorch Tensor, and AutoGrad (Variable is deprecated in the new version of pytorch). 
    - Understand Loss Functions

### What is PyTorch?
It's a Python based scientific computing package targeted as:
* A replacement for numpy to use the power of GPUs
* A deep learning research platform that provides maximum flexibility and speed

### Tensor
It is similar to Numpy Ndarray
<a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html">https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html 


In [35]:
from __future__ import print_function
import torch

#### Check Version of the Pytorch

In [36]:
print(torch.__version__)

1.0.0


#### Tensor Initialization

In [37]:
x = torch.Tensor(6, 2)  # construct a 6x2 matrix, uninitialized


In [38]:
x, x.size()

(tensor([[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]), torch.Size([6, 2]))

In [39]:
y = torch.rand(6, 2)  # construct a randomly initialized matrix


In [40]:
y, y.size()

(tensor([[0.4098, 0.2235],
         [0.4025, 0.6149],
         [0.7226, 0.2181],
         [0.1213, 0.0165],
         [0.9585, 0.6587],
         [0.4721, 0.9341]]), torch.Size([6, 2]))

In [41]:
z = torch.ones(7) # construct a matrix of ones


In [42]:
z, z.size()

(tensor([1., 1., 1., 1., 1., 1., 1.]), torch.Size([7]))

#### Operation Example: Addtion
Related reading and reference:
    
* PyTorch documentation:
<a href="http://pytorch.org/docs/0.3.0/"> http://pytorch.org/docs/0.3.0/ </a>


In [43]:
# addition: syntax 1
x + y

tensor([[0.4098, 0.2235],
        [0.4025, 0.6149],
        [0.7226, 0.2181],
        [0.1213, 0.0165],
        [0.9585, 0.6587],
        [0.4721, 0.9341]])

In [44]:
# addition: syntax 2
torch.add(x, y)

tensor([[0.4098, 0.2235],
        [0.4025, 0.6149],
        [0.7226, 0.2181],
        [0.1213, 0.0165],
        [0.9585, 0.6587],
        [0.4721, 0.9341]])

In [45]:
# addition: giving an output tensor
result = torch.Tensor(6, 2)
torch.add(x, y, out=result)

tensor([[0.4098, 0.2235],
        [0.4025, 0.6149],
        [0.7226, 0.2181],
        [0.1213, 0.0165],
        [0.9585, 0.6587],
        [0.4721, 0.9341]])

In [46]:
# addition: in-place
y.add_(x) # adds x to y

tensor([[0.4098, 0.2235],
        [0.4025, 0.6149],
        [0.7226, 0.2181],
        [0.1213, 0.0165],
        [0.9585, 0.6587],
        [0.4721, 0.9341]])

#### Numpy Bridge:
The torch Tensor and numpy array will share their underlying memory locations, and changing one will change the other.

##### Convert Torch Tensor to Numpy

In [47]:
a = torch.ones(5)
a

tensor([1., 1., 1., 1., 1.])

In [48]:
b = a.numpy()
b

array([1., 1., 1., 1., 1.], dtype=float32)

In [49]:
a.add_(1) # Remember this is an inplace addition
print(a)
print(b) # see how the numpy array changed in value

tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


##### Converting Numpy Array to Torch Tensor

In [50]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


####  Used of CUDA

In [51]:
# let us run this cell only if CUDA is available
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x + y

### Autograd: automatic differentiation
* The autograd package provides automatic differentiation for all operations on Tensors.It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

### Tensor
* torch.Tensor is the central class of the package. If you set its attribute .requires_grad as True, it starts to track all operations on it.
* When you finish your computation you can call .backward() and have all the gradients computed automatically.
* The gradient for this tensor will be accumulated into .grad attribute.
* To stop a tensor from tracking history, you can call .detach() to detach it from the computation history, and to prevent future computation from being tracked.

### Function
* Tensor and Function are interconnected and build up an acyclic graph, that encodes a complete history of computation.
* Each tensor has a .grad_fn attribute that references a Function that has created the Tensor (except for Tensors created by the user - their grad_fn is None).

Related Reading and Reference:
<a href="http://pytorch.org/docs/autograd"> http://pytorch.org/docs/autograd </a>

In [52]:
import torch

In [53]:
x = torch.ones((2, 2), requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [54]:
x 

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [55]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [56]:
print(y.grad_fn)

In [57]:
z = y * y * 2
out = z.mean()
print(z, out)

tensor([[18., 18.],
        [18., 18.]], grad_fn=<MulBackward0>) tensor(18., grad_fn=<MeanBackward1>)


In [58]:
# What's the gradient of X before backward() is performed?
print(x.grad)

None


In [59]:
# What's the correct gradient of X?

# .retain_grad() enables the .grad attribute of non-leaf variable. 
# Remember of the graph of this process? x is a leaf variable and y,z are non-leaf variables
# .backward() computes and accumulates gradient values w.r.t leaf variables 

y.retain_grad() 
out.backward()

print(x.grad)
print(y.grad)

# you should zero out the gradient after gradient calculation each time
# as backward() computes and accumulates gradient values
x.grad.data.zero_()
y.grad.data.zero_()

# Question: How do we get these values?

tensor([[3., 3.],
        [3., 3.]])
tensor([[3., 3.],
        [3., 3.]])


tensor([[0., 0.],
        [0., 0.]])

In [60]:
# Another way to calculate the gradient for more than one variables in the graph
from torch.autograd import grad

# set up the problem 
x = torch.ones((2, 2), requires_grad=True)
y = x + 2
z = y * y * 2
out = z.mean()

# torch.autograd.grad computes gradients of the output variables (out in this case) w.r.t input variables (x and y) 
# Please refer to pytorch documentation
grad(out, {x, y})

(tensor([[3., 3.],
         [3., 3.]]), tensor([[3., 3.],
         [3., 3.]]))

### Loss Functions

 Related Reference: 
<a href="http://pytorch.org/docs/master/nn.html#loss-functions">http://pytorch.org/docs/master/nn.html#loss-functions </a>

#### Mean Squared Error
Question: What is mean squared error? What are the inputs? What's the output?

In [61]:
import torch.nn as nn
input = torch.randn((4, 5), requires_grad=True)
target = torch.randn(4, 5)

In [62]:
print(input)
print(target)

tensor([[ 0.6143,  0.7347,  0.3239,  0.9107,  0.4076],
        [-0.3427,  1.1531,  0.5247, -0.7727, -0.7752],
        [ 1.6251, -0.7306,  1.4909,  0.1721, -0.3276],
        [-0.8661, -2.2872, -0.5532, -1.5455, -0.1715]], requires_grad=True)
tensor([[-2.1202,  0.0988,  0.3111,  1.0487, -2.0369],
        [ 0.6369,  1.2214,  0.0113,  2.3299, -0.5199],
        [-0.6184, -1.0727, -0.8911, -1.2253, -1.0886],
        [-0.8394, -0.4263, -2.1845,  0.2890,  0.9468]])


In [63]:
loss = nn.MSELoss()
output = loss(input, target) # Note, in actual training, input here will be replaced with the predicted values
output.backward()

In [64]:
output, input

(tensor(2.4446, grad_fn=<MseLossBackward>),
 tensor([[ 0.6143,  0.7347,  0.3239,  0.9107,  0.4076],
         [-0.3427,  1.1531,  0.5247, -0.7727, -0.7752],
         [ 1.6251, -0.7306,  1.4909,  0.1721, -0.3276],
         [-0.8661, -2.2872, -0.5532, -1.5455, -0.1715]], requires_grad=True))

#### Cross Entropy Loss
Question: What is cross entropy loss? What are the inputs? What's the output?

In [65]:
input = torch.randn((4, 5), requires_grad=True)
target = torch.LongTensor(4).random_(5)
print(input)
print(target)

tensor([[ 0.4317, -1.5261, -1.4464,  0.1479, -1.0624],
        [ 0.8452,  0.3932, -0.2030,  1.7839, -0.4970],
        [ 2.3479,  1.2203, -0.8254, -0.9851,  2.1366],
        [ 0.9958,  0.4802,  0.8173,  1.8425,  0.7706]], requires_grad=True)
tensor([2, 4, 1, 1])


In [66]:
# Filling in the code to calculated the cross-entropy loss
loss = nn.CrossEntropyLoss()
output = loss(input, target) # Note, in actual training, input here will be replaced with the predicted values
output

tensor(2.4408, grad_fn=<NllLossBackward>)

### Reference:
* Deep Learning with PyTorch: A 60 Minute Blitz:
    <a href="http://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html">http://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html
    
    
* PyTorch documentation:
<a href="http://pytorch.org/docs/0.3.0/"> http://pytorch.org/docs/0.3.0/ </a>